import all necessary package

In [251]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

use beautifulsoup to scrape the wiki page table

In [2]:
tor_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
rsp = requests.get(tor_url)

In [24]:
tor_soup = BeautifulSoup(rsp.content,'lxml')

In [27]:
table1 = tor_soup.find_all('table')[0]

In [29]:
data1 = pd.read_html(str(table1))

In [149]:
dataCode = data1[0]

dataCode is the data frame of the original data ,but need some clean up

## 1 data clean up

In [150]:
dataCode.columns = dataCode.loc[0]

In [151]:
dataCode.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [152]:
dataCode = dataCode.drop(0)

In [153]:
dataCode.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


remove all index with "not assigned" Borough

In [154]:
dataCode = dataCode.replace({'Borough':'Not assigned'},np.nan)

In [155]:
dataCode.head()

,Postcode,Borough,Neighbourhood
1,M1A,NaN,Not assigned
2,M2A,NaN,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [156]:
dataCode = dataCode.dropna(axis=0)

In [157]:
dataCode.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


if the Neighbourhood is not assigned, change it to the Borough value in the same index

In [197]:
dataCode.loc[dataCode['Neighbourhood']=='Not assigned','Neighbourhood'] = dataCode[dataCode['Neighbourhood']=='Not assigned']['Borough']

In [198]:
dataCode[dataCode['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


combine all neighbourhood the index with the same Borough and Postcode, use "," to seperate each

In [199]:
gp2 = dataCode.groupby('Postcode').agg({'Borough':'min','Neighbourhood':lambda x: ','.join(x)})

In [200]:
gp2.head(20)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


In [201]:
gp2.shape

(103, 2)

## 2 the lat/lon value collection

reset the index and give the final data2

In [202]:
data2 = pd.DataFrame(gp2)
data2.reset_index(inplace=True)
data2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [203]:
data2.shape

(103, 3)

the google map api is not working so use csv data downloaded to get the lat/lon value of each postcode

In [232]:
gps_data = pd.read_csv('./Geospatial_data')
gps_data = gps_data.rename(columns={'Postal Code':'Postcode'})
gps_data.set_index('Postcode',inplace=True)
gps_data.head(4)

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917


change the index of data2 to "Postcode" so that the gps data can be merged using the same index

In [224]:
data3 = data2.set_index('Postcode')

In [226]:
data3.head(5)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


data3 is the Borough/Neibourhood information of the toronto area, and gps value is the GPS lat/lon value of the same area
They can be merged with the same index value

In [242]:
data4= pd.concat([data3,gps_data],axis=1)

In [243]:
data4.head(10)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


use the postcode as the column and get the final data

In [244]:
data4 = data4.reset_index()

In [245]:
data4.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [246]:
data4.shape

(103, 5)

## 3 explore the neighbourhood


get the Canada center gps value

In [330]:
address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto is 43.653963, -79.387207.


In [328]:
data4.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [334]:
lat

43.706748299999994

In [338]:
map_canada = folium.Map(location=[latitude,longitude],zoom_start=12)
for lat, lng, borough, neighborhood in zip(data4['Latitude'], data4['Longitude'], data4['Borough'], data4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    folium.CircleMarker(
    location=[lat,lng],
    radius=5,
    popup=label,
        
    color='#3186cc',
    fill=True,
    fill_color='#3186cc',
        opacity=0.7
    ).add_to(map_canada)
    #print(lat,lng)
map_canada

In [335]:
map_canada = folium.Map(location=[latitude,longitude],zoom_start=12)
for lat, lng, borough, neighborhood in zip(data4['Latitude'], data4['Longitude'], data4['Borough'], data4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    folium.Marker(location=[lat, lng],
                  icon=folium.Icon(color='green'),
                  
                  
                 ).add_to(map_canada)
    #print(lat,lng)
map_canada

explore the neighbourhood using foursquare

In [285]:
CLIENT_ID = 'Q2J4GILEC3LE4DEIUP3VM2G5R3MUYI5PL2NW01KIV0Q4MCDN' # your Foursquare ID
CLIENT_SECRET = 'OFYFF2TGMXFEOTPE5IIECGADSZ0J1BYWTJU1CUNJZZOOWYL3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [369]:
data4.loc[10,:]

Postcode                                                       M1P
Borough                                                Scarborough
Neighbourhood    Dorset Park,Scarborough Town Centre,Wexford He...
Latitude                                                   43.7574
Longitude                                                 -79.2733
Name: 10, dtype: object

In [374]:
test_neighbourhodd = data4.loc[10,:]
neighborhood_latitude = data4.loc[10,:].Latitude
neighborhood_longitude = data4.loc[10,:].Longitude

In [372]:
radius = 500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [373]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c68b416dd57977bd6f6ff8f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Dorset Park',
  'headerFullLocation': 'Dorset Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.761909604500005,
    'lng': -79.26708530848985},
   'sw': {'lat': 43.7529095955, 'lng': -79.27952269151017}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5226562611d2cd49d83ef03b',
       'name': 'Kairali',
       'location': {'address': '1210 Kennedy Road',
        'crossStreet': 'Lawrence',
        'lat': 43.75476837158203,
        'lng': -79.2771987915039,
        'labeledLatLngs': [{'label': 'disp

In [375]:
test_neighbourhodd.Neighbourhood

'Dorset Park,Scarborough Town Centre,Wexford Heights'

In [384]:
print('there are {} neighbourhodd'.format(len(results['response']['groups'][0]['items'])))

there are 8 neighbourhodd


In [353]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighborhood_latitude, 
            neighborhood_longitude, 
            radius, 
            LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c681528db04f505a27e81d1'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [362]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [365]:
Toronto_venues = getNearbyVenues(names=data4['Neighbourhood'],
                                   latitudes=data4['Latitude'],
                                   longitudes=data4['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beac

In [366]:
Toronto_venues.shape

(2251, 7)

In [424]:
for each in Toronto_venues['Venue Category']:
    if each=='Neighborhodd':
        print(each)

In [385]:
Toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
5,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
6,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
8,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
9,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop


In [388]:
Toronto_venues.groupby('Neighborhood').count().sort_values(by=['Venue'],ascending =False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
St. James Town,100,100,100,100,100,100
"Ryerson,Garden District",100,100,100,100,100,100
"Harbourfront East,Toronto Islands,Union Station",100,100,100,100,100,100
"First Canadian Place,Underground city",100,100,100,100,100,100
"Design Exchange,Toronto Dominion Centre",100,100,100,100,100,100
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Stn A PO Boxes 25 The Esplanade,94,94,94,94,94,94


In [438]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


from the count there are 8 most crowded area , if it's up to me, then definatedly I would stay in Chinatown since I am Chinese. But let me try it harder 

### Analyze the data in Toronto

In [430]:
Toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
5,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
6,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
8,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
9,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop


In [470]:
for each in Toronto_venues['Venue Category']:
    if each=='Neighborhood':
        print(each)

Neighborhood
Neighborhood
Neighborhood
Neighborhood


it's interesting that there is a category named "Neighborhood", so to avoid ambigous, let's rename the real Neighborhood to "GeoNeighborhood"

In [442]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
old_column = list(Toronto_onehot.columns)
Toronto_onehot['GeoNeighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = ['GeoNeighborhood'] + old_column
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head(10)

,GeoNeighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [443]:
Toronto_onehot.shape

(2251, 280)

In [444]:
Toronto_grouped = Toronto_onehot.groupby('GeoNeighborhood').mean().reset_index()
Toronto_grouped

,GeoNeighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [445]:
Toronto_grouped.shape

(100, 280)

In [449]:
num_top_venues = 5

for hood in Toronto_grouped['GeoNeighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['GeoNeighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.loc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                       venue  freq
0             Clothing Store  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4  Middle Eastern Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2    Accessories Store   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.15
1           Pizza Place  0.08
2           Coffee Shop  0.08
3  Fast Food Restaurant  0.08
4              Pharmacy  0.08


----Alderwood,Long Branch----
  

                       venue  freq
0              Metro Station  0.25
1          Convenience Store  0.25
2                Coffee Shop  0.25
3                       Park  0.25
4  Middle Eastern Restaurant  0.00


----Emery,Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1                Accessories Store   0.0
2              Monument / Landmark   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.13
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.06
3                Bakery  0.05
4              Tea Room  0.03


----First Canadian Place,Underground city----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.08
2                Hotel  0.06
3           Restaurant  0.05
4  American Restaurant  0.04


----Flemingdon Park,Don Mills South----
              venue  freq
0       Cof

                       venue  freq
0                Coffee Shop  0.09
1             Clothing Store  0.09
2                       Café  0.04
3  Middle Eastern Restaurant  0.03
4             Cosmetics Shop  0.03


----Scarborough Village----
                 venue  freq
0           Playground   0.5
1           Smoke Shop   0.5
2    Accessories Store   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----Silver Hills,York Mills----
                       venue  freq
0                       Park   0.5
1                  Cafeteria   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4                      Motel   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2           Hotel  0.05
3            Café  0.05
4  Clothing Store  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.05
2                Café  0.04
3                 P

In [461]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [464]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['GeoNeighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Restaurant,Bakery,Hotel,Bar,Asian Restaurant
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Liquor Store,Pizza Place,Fried Chicken Joint,Discount Store,Sandwich Place,Coffee Shop,Beer Store,Fast Food Restaurant,Video Store
4,"Alderwood,Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Coffee Shop,Pub,Sandwich Place,Yoga Studio,Dog Run,Dessert Shop


### Cluster Neighborhoods

In [472]:
from sklearn.cluster import KMeans

In [473]:
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('GeoNeighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [475]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,2,Bar,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Dim Sum Restaurant
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run


In [477]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [479]:
Toronto_merged.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category',
       'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

In [481]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Neighborhood Latitude'], Toronto_merged['Neighborhood Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        #popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [483]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.806686,-79.199056,Fast Food Restaurant,0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
2,43.763573,-79.189914,Pizza Place,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run
3,43.763573,-79.191537,Electronics Store,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run
4,43.763573,-79.190720,Mexican Restaurant,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run
5,43.763573,-79.193406,Rental Car Location,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run
6,43.763573,-79.192286,Medical Center,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run
7,43.763573,-79.190466,Breakfast Spot,0,Mexican Restaurant,Rental Car Location,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Yoga Studio,Diner,Discount Store,Dog Run
8,43.770992,-79.221156,Coffee Shop,0,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
9,43.770992,-79.223078,Coffee Shop,0,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
10,43.770992,-79.214502,Korean Restaurant,0,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
